Welcome to the pyWWT visualization notebook!

This code is a use case for visualizing Galileo's magnetometer measurement campaign and demonstrates pyWWT's ability to visualize orbit data and overlay instrument data onto regions of interest.

To properly run this code, ensure that you have followed the steps outlined in the README.md file to pip install the appropriate APIs. Additionally, ensure that you have opened a "AAS Worldwide Telescope" launcher in your notebook work space. To do this simply go to File -> New Launcher and (if configured correctly) click the AAS WWT icon.

# Step 0:

Import necessary python libraries. 

In [1]:
import requests
import spiceypy
import pandas
import pandas as pd
import datetime
import os

from __future__ import print_function
from astropy.time import Time
from pywwt.jupyter import WWTJupyterWidget
from pywwt.jupyter import connect_to_app
from astropy.table import Table
# import pyWWTDevFunctions

import pathlib as Path


# Step 1:

Define Functions for script

In [2]:
def getsta(mag_time):
    METAKR = 'getgll.tm'
    spiceypy.furnsh( METAKR )
    row = []
    for t in mag_time:
            time_string = t.strftime('%Y-%m-%d %H:%M:%S')

            #ephemeris time after iterating
            et = spiceypy.str2et( time_string )

            [state, ltime] = spiceypy.spkezr( '5', et,      'J2000',
                                            'LT+S',   '-77'       )
            row.append([state[0], state[1], state[2], state[3], state[4], state[5]])
    return row

def get_data(url):
    tab_urls_cont = pandas.read_csv(url,sep = ',',header = None)
    tab_urls = [element for element in tab_urls_cont[1] if 'irc' in element]

    mag_time = []
    mag_tot = []
    for i in range(len(tab_urls)):
        time_data, iter_data = get_tabs(tab_urls[i]) # get rid of column in csv
        mag_time.extend(time_data)
        mag_tot.extend(iter_data)

    return mag_time, mag_tot

def get_tabs(url):
    #load lid found from parent collection
    lid_response = requests.get('https://pds.nasa.gov/api/search/1/products/'+url)
    lid_data = lid_response.json()

    time_data = []
    iter_data = []
    for i in range(1): #range(len(lid_data['data'])):
            tab_url = lid_data['properties']['ops:Data_File_Info.ops:file_ref'][0]
            #load in tab data
            tab_response = requests.get(tab_url)
            tab_content = tab_response.text
            lines = tab_content.splitlines()

            for line in lines:
                    raw_data = line.split()
                    time_data.append(raw_data[0])
                    iter_data.append(float(raw_data[4]))
    return time_data, iter_data


def download_and_load_spice_kernel(p_requests_session, p_url):
    if not os.access('.spice_kernels',os.F_OK):
        os.mkdir('.spice_kernels')

    kernel_dir = os.path.join(".spice_kernels")
#     os.makedirs(kernel_dir, exist_ok=True)
    file_path = os.path.join(kernel_dir, os.path.basename(p_url))
#     # TODO manage outdated local files

#     if not os.path.exists(file_path):
    r = requests.get(url, allow_redirects=True)
    if r.status_code == 200:
        print('Successfully loaded:',r.status_code)
#             log.debug(
#                 "copy kernel from %s to %s, status %s: %s",
#                 url,
#                 file_path,
#                 r.status_code,
#                 r.reason,
#             )
        open(file_path, "wb").write(r.content)
#     log.info("load spice kernel %s", file_path)
    spiceypy.furnsh(file_path)


# Step 2:

Run the pyWWTDevFunctions notebook in the background to load the custom functions built for this script. This step is mostly to cleanup the notebook and make it look nice. 

Import the necessary SPICE Kernels and download them to on the local environment. 

In [3]:

KERNEL_URLS = [
    #generic LSK
    "https://naif.jpl.nasa.gov/pub/naif/pds/data/ody-m-spice-6-v1.0/odsp_1000/data/lsk/naif0008.tls",
    #CASSINI Solar System SPK 
    "https://naif.jpl.nasa.gov/pub/naif/CASSINI/kernels/spk/981005_PLTEPH-DE405S.bsp",
    #Galileo SC SPK
    "https://naif.jpl.nasa.gov/pub/naif/GLL/kernels/spk/gll_951120_021126_raj2021.bsp"
]

requests_session = requests.session()
# requests_session.mount("file://", requests.adapters.LocalFileAdapter())
for url in KERNEL_URLS:
    download_and_load_spice_kernel(requests_session, url)
    
    

Successfully loaded: 200
Successfully loaded: 200
Successfully loaded: 200


# Step 3:

Utilize the PDS registry API to grab the magnetometer data for Galileo. 

In [4]:
url = "https://pds-ppi.igpp.ucla.edu/data/galileo-mag-jup-calibrated/data-highres-magnetosphere/collection-data-highres-magnetosphere.csv"
mag_time, mag_tot = get_data(url)

#Remove duplicates and maintain order of mag_data
sample = 100
df = pd.DataFrame({'time':mag_time[1::sample], 'data': mag_tot[1::sample]})
df['time'] = pd.to_datetime(df['time'])
#now sort values
df.sort_values('time', inplace = True)
df.drop_duplicates(subset='time', keep ='first', inplace = True)

df.reset_index(drop=True, inplace=True)

#Obtain the position data that corresponds to the measurements
print('Obtaining ephemeris for magnetometer data points...')
mag_ephem_data = getsta(df['time']) #Takes time input
print("Number of data points: "+ str(len(mag_ephem_data)))


#Fill in the gaps for the ephemeris data
print("Filling in ephemeris data...")
ephem_time = []
start_time = df['time'][0]

for i in range(3000):
        # start_time = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S")
        ephem_time.append(start_time + datetime.timedelta(minutes=i*1000))   
#Grab the states
ephem_data = getsta(ephem_time)

print('Finished')
        

Obtaining ephemeris for magnetometer data points...
Number of data points: 5111
Filling in ephemeris data...
Finished


# Step 4:

Fill in the ephemeris data for a specified sample size. This part is for visualizing the orbits of interest. 

In [5]:

        #Assign variables to table for pyWWT
ephem_tab = Table()
mag_tab = Table()

#Ephemeris data to table
ephem_iso = [ephem_t.replace(tzinfo=None).isoformat() for ephem_t in ephem_time]
ephem_tab['Time'] = ephem_iso

ephem_data = [row[0:3] for row in ephem_data]
transposed = zip(*ephem_data)
ephem_tab['X'], ephem_tab['Y'], ephem_tab['Z'] = transposed

#Magnetometer values and ephemeris data to table
mag_iso = [mag_t.replace(tzinfo=None).isoformat() for mag_t in df['time']]
mag_tab['Time'] = mag_iso
mag_val = [mag_t for mag_t in df['data']]
mag_tab['Magnitude (nT)'] = mag_val

mag_pos = [row[0:3] for row in mag_ephem_data]
mag_transpose = zip(*mag_pos)
mag_tab['X'], mag_tab['Y'], mag_tab['Z'] = mag_transpose
            
print('Done!')

Done!


In [6]:
#Connect pyWWT widget

wwt = WWTJupyterWidget()
wwt = await connect_to_app().becomes_ready()

In [7]:

wwt.reset()

wwt.set_current_time(Time('1995-12-07 15:21:01.776000'))
wwt.set_view("solar system")


In [8]:

layer = wwt.layers.add_table_layer(table = mag_tab, frame = 'jupiter',
                                  coord_type = 'rectangular',
                                  x_att = 'X', y_att = 'Y', z_att = 'Z')
layer.xyz_unit = 'km'
layer.far_side_visible = True


layer.cmap_att = 'Magnitude (nT)'
layer.cmap.vmin = min(mag_tab['Magnitude (nT)'] + 10)
layer.cmap.vmax = max(mag_tab['Magnitude (nT)'])

layer.size_scale = 1000
layer.far_side_visible = True
layer.marker_scale = 'world'

eph = wwt.layers.add_table_layer(table = ephem_tab, frame = 'jupiter',
                                  coord_type = 'rectangular',
                                  x_att = 'X', y_att = 'Y', z_att = 'Z')
eph.xyz_unit = 'km'
eph.far_side_visible = True
eph.color = (.4, .4, .6)
eph.size_scale = 16
eph.far_side_visible = True

wwt.solar_system.track_object('Jupiter')


In [26]:
#Animate

eph.time_series = True
eph.time_att = 'Time'
eph.time_decay = 1000000 * u.hour

layer.time_series = True
layer.time_att = 'Time'
layer.time_decay = 10000 * u.hour

my_time = Time('1996-12-07 15:21:01.776000')
wwt.set_current_time(my_time)

wwt.play_time(rate=10000)

In [27]:
wwt.pause_time()